# Teste quantificador do efeito de Tranfered Learning, Data Augmentation, Pre-Processamento e combinações destes na classificação de HIC

K = 10 </br>
Resnet50 </br>
Windowing sem transferencia de aprendizado </br>
Windowing com transferencia de aprendizado </br>
Windowing com transferencia de aprendizado e data aug</br>
Windowing sem transferencia de aprendizado e data aug</br>

## Imports e Installs

In [1]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Rescaling
from tensorflow.keras import optimizers
from tensorflow.keras.utils import image_dataset_from_directory

!pip install pydicom
import numpy as np
import pydicom
from PIL import Image
import os
import shutil

import cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 6.9 MB/s 


In [2]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

## Data Set

### Baixando

In [3]:
!kaggle datasets download -d pedroamaro/rsna-ds-w-35k-files-k-folders

100% 7.25G/7.25G [01:13<00:00, 174MB/s]
100% 7.25G/7.25G [01:13<00:00, 106MB/s]


In [4]:
!unzip /content/rsna-ds-w-35k-files-k-folders.zip | awk 'BEGIN {ORS=" "} {if(NR%1000==0)print "."}'
!rm /content/rsna-ds-w-35k-files-k-folders.zip

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [5]:
total_yes = 0
total_no = 0

for i in range(10):
    yesdir = "/content/k"+str(i)+"/yes/"
    nodir = "/content/k"+str(i)+"/no/"
    _, _, files = next(os.walk(yesdir))
    total_yes += len(files)
    _, _, files = next(os.walk(nodir))
    total_no += len(files)

print("Total de Imagens NO:")
print(total_no)
print("Total de Imagens YES:")
print(total_yes)

Total de Imagens NO:
30069
Total de Imagens YES:
4930


### Salvando como jpg

In [6]:
!mkdir jpg_k0
!mkdir jpg_k1
!mkdir jpg_k2
!mkdir jpg_k3
!mkdir jpg_k4
!mkdir jpg_k5
!mkdir jpg_k6
!mkdir jpg_k7
!mkdir jpg_k8
!mkdir jpg_k9

!mkdir jpg_k0/yes
!mkdir jpg_k1/yes
!mkdir jpg_k2/yes
!mkdir jpg_k3/yes
!mkdir jpg_k4/yes
!mkdir jpg_k5/yes
!mkdir jpg_k6/yes
!mkdir jpg_k7/yes
!mkdir jpg_k8/yes
!mkdir jpg_k9/yes

!mkdir jpg_k0/no
!mkdir jpg_k1/no
!mkdir jpg_k2/no
!mkdir jpg_k3/no
!mkdir jpg_k4/no
!mkdir jpg_k5/no
!mkdir jpg_k6/no
!mkdir jpg_k7/no
!mkdir jpg_k8/no
!mkdir jpg_k9/no

In [7]:
def segment_circle(windowed):
    original = windowed.copy().astype("uint8")
    mask = np.zeros(original.shape, dtype=np.uint8)
    gray = original
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=5)
    cnts = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        area = cv2.contourArea(c)
        if len(approx) > 4 and area > 10000 and area < 500000:
            ((x, y), r) = cv2.minEnclosingCircle(c)
            cv2.circle(mask, (int(x), int(y)), int(r), (255, 255, 255), -1)
            cv2.circle(original, (int(x), int(y)), int(r), (36, 255, 12), 0)
    x,y,w,h = cv2.boundingRect(mask)
    mask_ROI = mask[y:y+h, x:x+w]
    image_ROI = original[y:y+h, x:x+w]
    result = cv2.bitwise_and(image_ROI, image_ROI, mask=mask_ROI)
    return result, (x, y, w, h)

def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(img, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img


def preprocess_and_segment(path, make_correct=True):    
    dcm = pydicom.dcmread(path)
    if make_correct and (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)

    sample = dcm.pixel_array.astype("float32") * dcm.RescaleSlope + dcm.RescaleIntercept

    brain_window = window_image(sample, 40, 80)
    dural_window = window_image(sample, 80, 200)
    bone_window = window_image(sample, 600, 2800)

    brain_window, (x,y,w,h) = segment_circle(brain_window)
    if brain_window is None:
        return np.zeros((512, 512, 3))
    dural_window = dural_window[y:y+h, x:x+w]
    bone_window = bone_window[y:y+h, x:x+w]
    
    brain_window = (brain_window - (0.)) / 80.
    dural_window = (dural_window - (-20.)) / 200.
    bone_window = (bone_window - (-1200.)) / 2800.
    img_3ch = np.dstack([brain_window, dural_window, bone_window]).astype("float32")

    return img_3ch


def save_W_as_jpg(path, id, dest_folder):

  im = preprocess_and_segment(path)

  rescaled_im = (np.maximum(im, 0)/im.max())*255.0
  final_im = np.uint8(rescaled_im)

  final_im = Image.fromarray(final_im)
  final_im.save(dest_folder+id+'.jpg')

In [8]:
for i in range(10):
    dir = "/content/k"+str(i)+"/yes/"
    folder = os.listdir(dir)
    dest_folder = "/content/jpg_k"+str(i)+"/yes/"

    for id_ in folder:
      img_id = id_[:12]
      save_W_as_jpg(dir + f"{id_}", img_id, dest_folder)
      os.remove(dir + f"{id_}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide


In [9]:
for i in range(10):
    dir = "/content/k"+str(i)+"/no/"
    folder = os.listdir(dir)
    dest_folder = "/content/jpg_k"+str(i)+"/no/"

    for id_ in folder:
      img_id = id_[:12]
      save_W_as_jpg(dir + f"{id_}", img_id, dest_folder)
      os.remove(dir + f"{id_}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide


In [10]:
total_yes = 0
total_no = 0

for i in range(10):
    yesdir = "/content/jpg_k"+str(i)+"/yes/"
    nodir = "/content/jpg_k"+str(i)+"/no/"
    _, _, files = next(os.walk(yesdir))
    total_yes += len(files)
    _, _, files = next(os.walk(nodir))
    total_no += len(files)

print("Total de Imagens JPG EM NO:")
print(total_no)
print("Total de Imagens JPG EM YES:")
print(total_yes)

Total de Imagens JPG EM NO:
30069
Total de Imagens JPG EM YES:
4930


In [11]:
!rmdir k0/yes
!rmdir k1/yes
!rmdir k2/yes
!rmdir k3/yes
!rmdir k4/yes
!rmdir k5/yes
!rmdir k6/yes
!rmdir k7/yes
!rmdir k8/yes
!rmdir k9/yes

!rmdir k0/no
!rmdir k1/no
!rmdir k2/no
!rmdir k3/no
!rmdir k4/no
!rmdir k5/no
!rmdir k6/no
!rmdir k7/no
!rmdir k8/no
!rmdir k9/no

!rmdir k0
!rmdir k1
!rmdir k2
!rmdir k3
!rmdir k4
!rmdir k5
!rmdir k6
!rmdir k7
!rmdir k8
!rmdir k9

## Experimento

In [12]:
!mkdir train
!mkdir train/yes
!mkdir train/no

### Baseline: WO/ Tranfered Learning

#### Declarações

In [ ]:
def clear_train_dir():
  for file in os.listdir("/content/train/yes/"):
    os.remove("/content/train/yes/" + file)

  for file in os.listdir("/content/train/no/"):
    os.remove("/content/train/no/" + file)

In [ ]:
def load_train_dir(ki):
  k = [0,1,2,3,4,5,6,7,8,9]
  for i in range(10):
    if i!= k[ki]:
      yesdir = "/content/jpg_k"+str(i)+"/yes/"
      nodir = "/content/jpg_k"+str(i)+"/no/"
      for file in os.listdir(yesdir):
        shutil.copy(yesdir + file, "/content/train/yes")
      for file in os.listdir(nodir):
        shutil.copy(nodir + file, "/content/train/no")

In [ ]:
def train_model(w0, w1):
    train_data = image_dataset_from_directory(
      ("/content/train"),
      labels='inferred',
      validation_split=1/3,
      subset="training",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )
  
    val_data = image_dataset_from_directory(
      "/content/train",
      labels='inferred',
      validation_split=1/3,
      subset="validation",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )

    class_names = train_data.class_names

    normalization_layer = Rescaling(1./255)

    normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next(iter(normalized_data))

    normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
    val_image_batch, val_labels_batch = next(iter(normalized_val_data))

    keras.backend.clear_session()

    resnet_model = Sequential()

    resnet = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        input_shape=image_batch.shape[1:],
        pooling='avg',
        weights=None,
        classes=2,
    )

    resnet_model.add(resnet)

    resnet_model.add(Flatten())

    resnet_model.add(Dense(512, activation="relu"))

    resnet_model.add(Dropout(0.5))

    resnet_model.add(Dense(1, activation="sigmoid"))

    resnet_model.compile(loss='binary_crossentropy', 
                         optimizer=optimizers.Adam(learning_rate=0.00001),
                         metrics=[keras.metrics.AUC(), 'accuracy'])
  
    run_hist = resnet_model.fit(
      normalized_data, 
      validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1}, 
      callbacks=[keras.callbacks.EarlyStopping(
                      monitor="val_auc",
                      min_delta=0,
                      patience=5,
                      verbose=1,
                      mode="auto",
                      baseline=None,
                      restore_best_weights=True,
                  )]
    )

    return resnet_model

In [ ]:
def test_model(resnet_model, test_path):
    test_data = image_dataset_from_directory(
      test_path,
      labels='inferred',
      label_mode="int",
      batch_size = 64,
      color_mode = "rgb"
    )

    normalization_layer = Rescaling(1./255)

    normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
    test_image_batch, test_labels_batch = next(iter(normalized_test_data))

    results = resnet_model.evaluate(normalized_test_data)
    return results

In [ ]:
neg = 27062
pos = 4437
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
print('Numero de imagens: NO')
!ls /content/train/no | wc -l
print('Numero de imagens: YES')
!ls /content/train/yes | wc -l

Numero de imagens: NO
0
Numero de imagens: YES
0


#### k0

In [ ]:
i = 0
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 323s 924ms/step - loss: 0.7328 - auc: 0.6273 - accuracy: 0.5764 - val_loss: 0.6793 - val_auc: 0.3876 - val_accuracy: 0.6743
Epoch 2/20
329/329 [==============================] - 311s 942ms/step - loss: 0.6405 - auc: 0.7016 - accuracy: 0.6150 - val_loss: 0.7786 - val_auc: 0.7175 - val_accuracy: 0.4648
Epoch 3/20
329/329 [==============================] - 312s 946ms/step - loss: 0.6106 - auc: 0.7342 - accuracy: 0.6406 - val_loss: 0.5440 - val_auc: 0.7557 - val_accuracy: 0.7226
Epoch 4/20
329/329 [==============================] - 312s 946ms/step - loss: 0.5853 - auc: 0.7578 - accuracy: 0.6724 - val_loss: 0.5569 - val_auc: 0.7781 - val_accuracy: 0.7103
Epoch 5/20
329/329 [==============================] - 311s 943ms/step - loss: 0.5588 - auc: 0.7849 - accuracy: 0.7059 - val_loss: 0.6053 -

In [ ]:
clear_train_dir()

####k1

In [ ]:
i = 1
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 318s 949ms/step - loss: 0.7426 - auc: 0.6295 - accuracy: 0.5795 - val_loss: 0.7870 - val_auc: 0.6108 - val_accuracy: 0.1552
Epoch 2/20
329/329 [==============================] - 349s 1s/step - loss: 0.6329 - auc: 0.7107 - accuracy: 0.6209 - val_loss: 0.7495 - val_auc: 0.7278 - val_accuracy: 0.5152
Epoch 3/20
329/329 [==============================] - 345s 1s/step - loss: 0.5964 - auc: 0.7454 - accuracy: 0.6520 - val_loss: 0.6935 - val_auc: 0.7612 - val_accuracy: 0.6045
Epoch 4/20
329/329 [==============================] - 311s 942ms/step - loss: 0.5840 - auc: 0.7607 - accuracy: 0.6676 - val_loss: 0.6608 - val_auc: 0.7642 - val_accuracy: 0.6248
Epoch 5/20
329/329 [==============================] - 311s 943ms/step - loss: 0.5536 - auc: 0.7892 - accuracy: 0.7037 - val_loss: 0.7655 - val_a

In [ ]:
clear_train_dir()

####k2

In [ ]:
i = 2
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 327s 937ms/step - loss: 0.7304 - auc: 0.6249 - accuracy: 0.5750 - val_loss: 0.9373 - val_auc: 0.7000 - val_accuracy: 0.1554
Epoch 2/100
329/329 [==============================] - 313s 950ms/step - loss: 0.6362 - auc: 0.7099 - accuracy: 0.6237 - val_loss: 0.8878 - val_auc: 0.6563 - val_accuracy: 0.3893
Epoch 3/100
329/329 [==============================] - 313s 950ms/step - loss: 0.6054 - auc: 0.7374 - accuracy: 0.6443 - val_loss: 0.6536 - val_auc: 0.7623 - val_accuracy: 0.6389
Epoch 4/100
329/329 [==============================] - 313s 949ms/step - loss: 0.5896 - auc: 0.7532 - accuracy: 0.6650 - val_loss: 0.4939 - val_auc: 0.7182 - val_accuracy: 0.6476
Epoch 5/100
329/329 [==============================] - 351s 1s/step - loss: 0.5674 - auc: 0.7762 - accuracy: 0.6885 - val_loss: 0.5927

In [ ]:
clear_train_dir()

####k3

In [ ]:
i = 3
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 345s 1s/step - loss: 0.7093 - auc: 0.6410 - accuracy: 0.5841 - val_loss: 1.4652 - val_auc: 0.6839 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 343s 1s/step - loss: 0.6268 - auc: 0.7125 - accuracy: 0.6203 - val_loss: 0.9108 - val_auc: 0.7294 - val_accuracy: 0.2913
Epoch 3/100
329/329 [==============================] - 304s 923ms/step - loss: 0.5959 - auc: 0.7441 - accuracy: 0.6390 - val_loss: 0.6879 - val_auc: 0.7608 - val_accuracy: 0.6017
Epoch 4/100
329/329 [==============================] - 308s 933ms/step - loss: 0.5870 - auc: 0.7558 - accuracy: 0.6603 - val_loss: 0.6012 - val_auc: 0.7671 - val_accuracy: 0.6511
Epoch 5/100
329/329 [==============================] - 307s 930ms/step - loss: 0.5744 - auc: 0.7687 - accuracy: 0.6745 - val_loss: 0.8575 - 

In [ ]:
clear_train_dir()

####k4

In [ ]:
i = 4
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 322s 923ms/step - loss: 0.7222 - auc: 0.6410 - accuracy: 0.5858 - val_loss: 1.1072 - val_auc: 0.6918 - val_accuracy: 0.1417
Epoch 2/100
329/329 [==============================] - 311s 943ms/step - loss: 0.6308 - auc: 0.7147 - accuracy: 0.6318 - val_loss: 0.6246 - val_auc: 0.7409 - val_accuracy: 0.6219
Epoch 3/100
329/329 [==============================] - 311s 944ms/step - loss: 0.5958 - auc: 0.7465 - accuracy: 0.6505 - val_loss: 0.5815 - val_auc: 0.7533 - val_accuracy: 0.6690
Epoch 4/100
329/329 [==============================] - 311s 943ms/step - loss: 0.5813 - auc: 0.7623 - accuracy: 0.6757 - val_loss: 0.6494 - val_auc: 0.7711 - val_accuracy: 0.6367
Epoch 5/100
329/329 [==============================] - 310s 941ms/step - loss: 0.5564 - auc: 0.7857 - accuracy: 0.7031 - val_loss: 0.7

In [ ]:
clear_train_dir()

####k5

In [ ]:
i = 5
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 315s 902ms/step - loss: 0.7291 - auc: 0.6376 - accuracy: 0.5833 - val_loss: 0.9893 - val_auc: 0.6636 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 300s 911ms/step - loss: 0.6411 - auc: 0.7015 - accuracy: 0.6170 - val_loss: 0.6252 - val_auc: 0.7421 - val_accuracy: 0.6325
Epoch 3/100
329/329 [==============================] - 300s 911ms/step - loss: 0.6055 - auc: 0.7384 - accuracy: 0.6484 - val_loss: 0.6835 - val_auc: 0.7663 - val_accuracy: 0.5941
Epoch 4/100
329/329 [==============================] - 340s 1s/step - loss: 0.5820 - auc: 0.7610 - accuracy: 0.6724 - val_loss: 0.8330 - val_auc: 0.7725 - val_accuracy: 0.5250
Epoch 5/100
329/329 [==============================] - 300s 910ms/step - loss: 0.5506 - auc: 0.7908 - accuracy: 0.7081 - val_loss: 0.4542

In [ ]:
clear_train_dir()

####k6

In [ ]:
i = 6
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 306s 913ms/step - loss: 0.7308 - auc: 0.6259 - accuracy: 0.5754 - val_loss: 0.8740 - val_auc: 0.5602 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 300s 911ms/step - loss: 0.6377 - auc: 0.7016 - accuracy: 0.6115 - val_loss: 0.5942 - val_auc: 0.7392 - val_accuracy: 0.6423
Epoch 3/100
329/329 [==============================] - 300s 911ms/step - loss: 0.6084 - auc: 0.7343 - accuracy: 0.6290 - val_loss: 0.5940 - val_auc: 0.7631 - val_accuracy: 0.6435
Epoch 4/100
329/329 [==============================] - 300s 911ms/step - loss: 0.5870 - auc: 0.7554 - accuracy: 0.6624 - val_loss: 0.4842 - val_auc: 0.7735 - val_accuracy: 0.7596
Epoch 5/100
329/329 [==============================] - 300s 911ms/step - loss: 0.5693 - auc: 0.7739 - accuracy: 0.6906 - val_loss: 0.5

In [ ]:
clear_train_dir()

####k7

In [ ]:
i = 7
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 318s 910ms/step - loss: 0.7196 - auc: 0.6457 - accuracy: 0.5835 - val_loss: 1.4993 - val_auc: 0.5963 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 301s 912ms/step - loss: 0.6319 - auc: 0.7150 - accuracy: 0.6268 - val_loss: 0.7920 - val_auc: 0.7423 - val_accuracy: 0.5190
Epoch 3/100
329/329 [==============================] - 301s 912ms/step - loss: 0.6025 - auc: 0.7435 - accuracy: 0.6506 - val_loss: 0.8218 - val_auc: 0.7665 - val_accuracy: 0.5362
Epoch 4/100
329/329 [==============================] - 301s 912ms/step - loss: 0.5761 - auc: 0.7691 - accuracy: 0.6826 - val_loss: 0.6039 - val_auc: 0.7817 - val_accuracy: 0.6655
Epoch 5/100
329/329 [==============================] - 300s 910ms/step - loss: 0.5462 - auc: 0.7948 - accuracy: 0.7097 - val_loss: 0.5

In [ ]:
clear_train_dir()

####k8

In [ ]:
i = 8
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 306s 911ms/step - loss: 0.7247 - auc: 0.6334 - accuracy: 0.5757 - val_loss: 0.6324 - val_auc: 0.6741 - val_accuracy: 0.7104
Epoch 2/100
329/329 [==============================] - 300s 910ms/step - loss: 0.6367 - auc: 0.7003 - accuracy: 0.6067 - val_loss: 0.7613 - val_auc: 0.7310 - val_accuracy: 0.4142
Epoch 3/100
329/329 [==============================] - 300s 909ms/step - loss: 0.6051 - auc: 0.7373 - accuracy: 0.6380 - val_loss: 0.6614 - val_auc: 0.7605 - val_accuracy: 0.5980
Epoch 4/100
329/329 [==============================] - 300s 911ms/step - loss: 0.5873 - auc: 0.7578 - accuracy: 0.6663 - val_loss: 0.5396 - val_auc: 0.7702 - val_accuracy: 0.6945
Epoch 5/100
329/329 [==============================] - 300s 911ms/step - loss: 0.5612 - auc: 0.7817 - accuracy: 0.6973 - val_loss: 0.4

In [ ]:
clear_train_dir()

####k9

In [ ]:
i = 9
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31500 files belonging to 2 classes.
Using 21000 files for training.
Found 31500 files belonging to 2 classes.
Using 10500 files for validation.
Epoch 1/100
329/329 [==============================] - 320s 913ms/step - loss: 0.7350 - auc: 0.6325 - accuracy: 0.5877 - val_loss: 1.0821 - val_auc: 0.4968 - val_accuracy: 0.1412
Epoch 2/100
329/329 [==============================] - 302s 916ms/step - loss: 0.6376 - auc: 0.7037 - accuracy: 0.6203 - val_loss: 0.7637 - val_auc: 0.7199 - val_accuracy: 0.4043
Epoch 3/100
329/329 [==============================] - 302s 917ms/step - loss: 0.6053 - auc: 0.7368 - accuracy: 0.6464 - val_loss: 0.6304 - val_auc: 0.7538 - val_accuracy: 0.6219
Epoch 4/100
329/329 [==============================] - 302s 916ms/step - loss: 0.5820 - auc: 0.7612 - accuracy: 0.6716 - val_loss: 0.6037 - val_auc: 0.7727 - val_accuracy: 0.6532
Epoch 5/100
329/329 [==============================] - 302s 917ms/step - loss: 0.5617 - auc: 0.7821 - accuracy: 0.6940 - val_loss: 0.5

In [ ]:
clear_train_dir()

### Baseline: W/ Tranfered Learning & WO/ Data Aug

#### Declarações

In [ ]:
def clear_train_dir():
  for file in os.listdir("/content/train/yes/"):
    os.remove("/content/train/yes/" + file)

  for file in os.listdir("/content/train/no/"):
    os.remove("/content/train/no/" + file)

In [ ]:
def load_train_dir(ki):
  k = [0,1,2,3,4,5,6,7,8,9]
  for i in range(10):
    if i!= k[ki]:
      yesdir = "/content/jpg_k"+str(i)+"/yes/"
      nodir = "/content/jpg_k"+str(i)+"/no/"
      for file in os.listdir(yesdir):
        shutil.copy(yesdir + file, "/content/train/yes")
      for file in os.listdir(nodir):
        shutil.copy(nodir + file, "/content/train/no")

In [ ]:
def train_model(w0, w1):
    train_data = image_dataset_from_directory(
      ("/content/train"),
      labels='inferred',
      validation_split=1/3,
      subset="training",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )
  
    val_data = image_dataset_from_directory(
      "/content/train",
      labels='inferred',
      validation_split=1/3,
      subset="validation",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )

    class_names = train_data.class_names

    normalization_layer = Rescaling(1./255)

    normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next(iter(normalized_data))

    normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
    val_image_batch, val_labels_batch = next(iter(normalized_val_data))

    keras.backend.clear_session()

    resnet_model = Sequential()

    resnet = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        input_shape=image_batch.shape[1:],
        pooling='avg',
        weights='imagenet',
        classes=2,
    )

    resnet_model.add(resnet)

    resnet_model.add(Flatten())

    resnet_model.add(Dense(512, activation="relu"))

    resnet_model.add(Dropout(0.5))

    resnet_model.add(Dense(1, activation="sigmoid"))

    resnet_model.compile(loss='binary_crossentropy', 
                         optimizer=optimizers.Adam(learning_rate=0.00001),
                         metrics=[keras.metrics.AUC(), 'accuracy'])
  
    run_hist = resnet_model.fit(
      normalized_data, 
      validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1}, 
      callbacks=[keras.callbacks.EarlyStopping(
                      monitor="val_auc",
                      min_delta=0,
                      patience=5,
                      verbose=1,
                      mode="auto",
                      baseline=None,
                      restore_best_weights=True,
                  )]
    )

    return resnet_model

In [ ]:
def test_model(resnet_model, test_path):
    test_data = image_dataset_from_directory(
      test_path,
      labels='inferred',
      label_mode="int",
      batch_size = 64,
      color_mode = "rgb"
    )

    normalization_layer = Rescaling(1./255)

    normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
    test_image_batch, test_labels_batch = next(iter(normalized_test_data))

    results = resnet_model.evaluate(normalized_test_data)
    return results

In [ ]:
neg = 27062
pos = 4437
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
print('Numero de imagens: NO')
!ls /content/train/no | wc -l
print('Numero de imagens: YES')
!ls /content/train/yes | wc -l

Numero de imagens: NO
0
Numero de imagens: YES
0


#### k0

In [ ]:
i = 0
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 2s 0us/step
Epoch 1/100
329/329 [==============================] - 309s 919ms/step - loss: 0.5590 - auc: 0.7873 - accuracy: 0.7253 - val_loss: 0.5151 - val_auc: 0.5174 - val_accuracy: 0.8448
Epoch 2/100
329/329 [==============================] - 301s 914ms/step - loss: 0.4020 - auc: 0.8960 - accuracy: 0.8468 - val_loss: 0.4818 - val_auc: 0.5607 - val_accuracy: 0.8217
Epoch 3/100
329/329 [==============================] - 302s 915ms/step - loss: 0.3051 - auc: 0.9423 - accuracy: 0.8968 - val_loss: 0.4758 - val_auc: 0.8572 - val_accuracy: 0.8235
Epoch 4/100
329/329 [==============================] - 342s 1s/step - loss: 0.2132 - auc: 0.9722 - accuracy: 0.9385 - val_loss: 0.3793 - val_auc: 0.8832 - val_accuracy: 0.8776
Epoch 5/100
329/329 [==============================] - 301s 913ms/step -

In [ ]:
clear_train_dir()

####k1

In [ ]:
i = 1
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 349s 1s/step - loss: 0.5724 - auc: 0.7766 - accuracy: 0.6996 - val_loss: 0.4552 - val_auc: 0.5283 - val_accuracy: 0.8557
Epoch 2/100
329/329 [==============================] - 302s 915ms/step - loss: 0.4125 - auc: 0.8907 - accuracy: 0.8307 - val_loss: 0.4477 - val_auc: 0.5803 - val_accuracy: 0.8608
Epoch 3/100
329/329 [==============================] - 342s 1s/step - loss: 0.3116 - auc: 0.9396 - accuracy: 0.8854 - val_loss: 0.4114 - val_auc: 0.8282 - val_accuracy: 0.8554
Epoch 4/100
329/329 [==============================] - 302s 915ms/step - loss: 0.2197 - auc: 0.9711 - accuracy: 0.9312 - val_loss: 0.3448 - val_auc: 0.8703 - val_accuracy: 0.8929
Epoch 5/100
329/329 [==============================] - 301s 914ms/step - loss: 0.1551 - auc: 0.9841 - accuracy: 0.9594 - val_loss: 0.3565 - 

In [ ]:
clear_train_dir()

####k2

In [ ]:
i = 2
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 310s 922ms/step - loss: 0.5772 - auc: 0.7740 - accuracy: 0.7077 - val_loss: 0.5803 - val_auc: 0.4892 - val_accuracy: 0.7972
Epoch 2/100
329/329 [==============================] - 342s 1s/step - loss: 0.4035 - auc: 0.8964 - accuracy: 0.8421 - val_loss: 0.5373 - val_auc: 0.7058 - val_accuracy: 0.8107
Epoch 3/100
329/329 [==============================] - 303s 918ms/step - loss: 0.3106 - auc: 0.9406 - accuracy: 0.8899 - val_loss: 0.5578 - val_auc: 0.8558 - val_accuracy: 0.7953
Epoch 4/100
329/329 [==============================] - 302s 918ms/step - loss: 0.2240 - auc: 0.9697 - accuracy: 0.9305 - val_loss: 0.3766 - val_auc: 0.8895 - val_accuracy: 0.8780
Epoch 5/100
329/329 [==============================] - 303s 918ms/step - loss: 0.1520 - auc: 0.9849 - accuracy: 0.9620 - val_loss: 0.3749

In [ ]:
clear_train_dir()

####k3

In [ ]:
i = 3
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 311s 924ms/step - loss: 0.5712 - auc: 0.7783 - accuracy: 0.7024 - val_loss: 0.4829 - val_auc: 0.5904 - val_accuracy: 0.8564
Epoch 2/100
329/329 [==============================] - 303s 918ms/step - loss: 0.4039 - auc: 0.8954 - accuracy: 0.8347 - val_loss: 0.4607 - val_auc: 0.6147 - val_accuracy: 0.8444
Epoch 3/100
329/329 [==============================] - 303s 920ms/step - loss: 0.3042 - auc: 0.9427 - accuracy: 0.8893 - val_loss: 0.4421 - val_auc: 0.8315 - val_accuracy: 0.8459
Epoch 4/100
329/329 [==============================] - 303s 920ms/step - loss: 0.2146 - auc: 0.9718 - accuracy: 0.9334 - val_loss: 0.3544 - val_auc: 0.8696 - val_accuracy: 0.8981
Epoch 5/100
329/329 [==============================] - 303s 919ms/step - loss: 0.1550 - auc: 0.9844 - accuracy: 0.9599 - val_loss: 0.3

In [ ]:
clear_train_dir()

####k4

In [ ]:
i = 4
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/100
329/329 [==============================] - 317s 907ms/step - loss: 0.5636 - auc: 0.7832 - accuracy: 0.7149 - val_loss: 0.6227 - val_auc: 0.6077 - val_accuracy: 0.5677
Epoch 2/100
329/329 [==============================] - 301s 914ms/step - loss: 0.4111 - auc: 0.8913 - accuracy: 0.8380 - val_loss: 0.4457 - val_auc: 0.6604 - val_accuracy: 0.8663
Epoch 3/100
329/329 [==============================] - 301s 913ms/step - loss: 0.3152 - auc: 0.9382 - accuracy: 0.8897 - val_loss: 0.5926 - val_auc: 0.8614 - val_accuracy: 0.7827
Epoch 4/100
329/329 [==============================] - 301s 914ms/step - loss: 0.2231 - auc: 0.9703 - accuracy: 0.9335 - val_loss: 0.3482 - val_auc: 0.8912 - val_accuracy: 0.8899
Epoch 5/100
329/329 [==============================] - 341s 1s/step -

In [ ]:
clear_train_dir()

####k5

In [ ]:
i = 5
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 308s 917ms/step - loss: 0.5810 - auc: 0.7716 - accuracy: 0.7063 - val_loss: 0.4886 - val_auc: 0.6113 - val_accuracy: 0.8447
Epoch 2/100
329/329 [==============================] - 341s 1s/step - loss: 0.4117 - auc: 0.8915 - accuracy: 0.8328 - val_loss: 0.3612 - val_auc: 0.7424 - val_accuracy: 0.8695
Epoch 3/100
329/329 [==============================] - 301s 913ms/step - loss: 0.3150 - auc: 0.9386 - accuracy: 0.8857 - val_loss: 0.3992 - val_auc: 0.8675 - val_accuracy: 0.8523
Epoch 4/100
329/329 [==============================] - 301s 913ms/step - loss: 0.2225 - auc: 0.9699 - accuracy: 0.9298 - val_loss: 0.3274 - val_auc: 0.8818 - val_accuracy: 0.8934
Epoch 5/100
329/329 [==============================] - 301s 913ms/step - loss: 0.1530 - auc: 0.9848 - accuracy: 0.9600 - val_loss: 0.3897

In [ ]:
clear_train_dir()

####k6

In [ ]:
i = 6
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 349s 1s/step - loss: 0.5622 - auc: 0.7863 - accuracy: 0.7210 - val_loss: 0.6747 - val_auc: 0.6100 - val_accuracy: 0.5865
Epoch 2/100
329/329 [==============================] - 301s 914ms/step - loss: 0.3984 - auc: 0.8981 - accuracy: 0.8478 - val_loss: 0.8525 - val_auc: 0.6677 - val_accuracy: 0.4692
Epoch 3/100
329/329 [==============================] - 302s 915ms/step - loss: 0.3053 - auc: 0.9416 - accuracy: 0.8954 - val_loss: 0.6522 - val_auc: 0.8507 - val_accuracy: 0.7601
Epoch 4/100
329/329 [==============================] - 302s 915ms/step - loss: 0.2157 - auc: 0.9712 - accuracy: 0.9357 - val_loss: 0.3419 - val_auc: 0.8839 - val_accuracy: 0.8944
Epoch 5/100
329/329 [==============================] - 341s 1s/step - loss: 0.1510 - auc: 0.9845 - accuracy: 0.9629 - val_loss: 0.3655 - 

In [ ]:
clear_train_dir()

####k7

In [ ]:
i = 7
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 308s 918ms/step - loss: 0.5645 - auc: 0.7833 - accuracy: 0.7065 - val_loss: 0.4953 - val_auc: 0.4244 - val_accuracy: 0.8576
Epoch 2/100
329/329 [==============================] - 301s 914ms/step - loss: 0.4142 - auc: 0.8890 - accuracy: 0.8320 - val_loss: 0.4016 - val_auc: 0.6529 - val_accuracy: 0.8618
Epoch 3/100
329/329 [==============================] - 302s 915ms/step - loss: 0.3116 - auc: 0.9402 - accuracy: 0.8944 - val_loss: 0.3987 - val_auc: 0.8557 - val_accuracy: 0.8464
Epoch 4/100
329/329 [==============================] - 301s 914ms/step - loss: 0.2136 - auc: 0.9727 - accuracy: 0.9380 - val_loss: 0.4168 - val_auc: 0.8883 - val_accuracy: 0.8652
Epoch 5/100
329/329 [==============================] - 341s 1s/step - loss: 0.1458 - auc: 0.9862 - accuracy: 0.9657 - val_loss: 0.4007

In [ ]:
clear_train_dir()

####k8

In [ ]:
i = 8
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/100
329/329 [==============================] - 319s 916ms/step - loss: 0.5696 - auc: 0.7811 - accuracy: 0.7157 - val_loss: 0.4441 - val_auc: 0.4504 - val_accuracy: 0.8599
Epoch 2/100
329/329 [==============================] - 346s 1s/step - loss: 0.4055 - auc: 0.8933 - accuracy: 0.8375 - val_loss: 0.3933 - val_auc: 0.6451 - val_accuracy: 0.8604
Epoch 3/100
329/329 [==============================] - 309s 936ms/step - loss: 0.3054 - auc: 0.9420 - accuracy: 0.8934 - val_loss: 0.4730 - val_auc: 0.8492 - val_accuracy: 0.8183
Epoch 4/100
329/329 [==============================] - 308s 934ms/step - loss: 0.2167 - auc: 0.9715 - accuracy: 0.9338 - val_loss: 0.3501 - val_auc: 0.8832 - val_accuracy: 0.8832
Epoch 5/100
329/329 [==============================] - 307s 933ms/step -

In [ ]:
clear_train_dir()

####k9

In [ ]:
i = 9
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31500 files belonging to 2 classes.
Using 21000 files for training.
Found 31500 files belonging to 2 classes.
Using 10500 files for validation.
Epoch 1/100
329/329 [==============================] - 314s 936ms/step - loss: 0.5485 - auc: 0.7964 - accuracy: 0.7264 - val_loss: 0.4473 - val_auc: 0.5181 - val_accuracy: 0.8588
Epoch 2/100
329/329 [==============================] - 309s 936ms/step - loss: 0.3984 - auc: 0.8984 - accuracy: 0.8449 - val_loss: 0.4284 - val_auc: 0.6328 - val_accuracy: 0.8363
Epoch 3/100
329/329 [==============================] - 346s 1s/step - loss: 0.2984 - auc: 0.9445 - accuracy: 0.8953 - val_loss: 0.6105 - val_auc: 0.8601 - val_accuracy: 0.7545
Epoch 4/100
329/329 [==============================] - 307s 931ms/step - loss: 0.2131 - auc: 0.9718 - accuracy: 0.9348 - val_loss: 0.3872 - val_auc: 0.8832 - val_accuracy: 0.8755
Epoch 5/100
329/329 [==============================] - 307s 932ms/step - loss: 0.1488 - auc: 0.9854 - accuracy: 0.9630 - val_loss: 0.3658

In [ ]:
clear_train_dir()

### Baseline: W/ Tranfered Learning & W/ Data Aug

In [ ]:
from keras.layers import RandomFlip, RandomRotation, RandomCrop

#### Declarações

In [ ]:
def clear_train_dir():
  for file in os.listdir("/content/train/yes/"):
    os.remove("/content/train/yes/" + file)

  for file in os.listdir("/content/train/no/"):
    os.remove("/content/train/no/" + file)

In [ ]:
def load_train_dir(ki):
  k = [0,1,2,3,4,5,6,7,8,9]
  for i in range(10):
    if i!= k[ki]:
      yesdir = "/content/jpg_k"+str(i)+"/yes/"
      nodir = "/content/jpg_k"+str(i)+"/no/"
      for file in os.listdir(yesdir):
        shutil.copy(yesdir + file, "/content/train/yes")
      for file in os.listdir(nodir):
        shutil.copy(nodir + file, "/content/train/no")

In [ ]:
def train_model(w0, w1):
    train_data = image_dataset_from_directory(
      ("/content/train"),
      labels='inferred',
      validation_split=1/3,
      subset="training",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )
  
    val_data = image_dataset_from_directory(
      "/content/train",
      labels='inferred',
      validation_split=1/3,
      subset="validation",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )

    class_names = train_data.class_names

    normalization_layer = Rescaling(1./255)

    normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next(iter(normalized_data))

    normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
    val_image_batch, val_labels_batch = next(iter(normalized_val_data))

    keras.backend.clear_session()

    resnet_model = Sequential()

    resnet_model.add(RandomFlip("horizontal"))
    resnet_model.add(RandomRotation(0.2))
    resnet_model.add(RandomCrop(256, 256, seed=1)) 

    resnet = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        input_shape=image_batch.shape[1:],
        pooling='avg',
        weights='imagenet',
        classes=2,
    )

    resnet_model.add(resnet)

    resnet_model.add(Flatten())

    resnet_model.add(Dense(512, activation="relu"))

    resnet_model.add(Dropout(0.5))

    resnet_model.add(Dense(1, activation="sigmoid"))

    resnet_model.compile(loss='binary_crossentropy', 
                         optimizer=optimizers.Adam(learning_rate=0.00001),
                         metrics=[keras.metrics.AUC(), 'accuracy'])
  
    run_hist = resnet_model.fit(
      normalized_data, 
      validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1}, 
      callbacks=[keras.callbacks.EarlyStopping(
                      monitor="val_auc",
                      min_delta=0,
                      patience=5,
                      verbose=1,
                      mode="auto",
                      baseline=None,
                      restore_best_weights=True,
                  )]
    )

    return resnet_model

In [ ]:
def test_model(resnet_model, test_path):
    test_data = image_dataset_from_directory(
      test_path,
      labels='inferred',
      label_mode="int",
      batch_size = 64,
      color_mode = "rgb"
    )

    normalization_layer = Rescaling(1./255)

    normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
    test_image_batch, test_labels_batch = next(iter(normalized_test_data))

    results = resnet_model.evaluate(normalized_test_data)
    return results

In [ ]:
neg = 27062
pos = 4437
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

In [ ]:
print('Numero de imagens: NO')
!ls /content/train/no | wc -l
print('Numero de imagens: YES')
!ls /content/train/yes | wc -l

#### k0

In [ ]:
i = 0
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 372s 1s/step - loss: 0.6037 - auc: 0.7470 - accuracy: 0.6805 - val_loss: 0.4958 - val_auc: 0.4362 - val_accuracy: 0.8587
Epoch 2/100
329/329 [==============================] - 326s 989ms/step - loss: 0.4824 - auc: 0.8465 - accuracy: 0.7900 - val_loss: 0.5386 - val_auc: 0.6451 - val_accuracy: 0.7812
Epoch 3/100
329/329 [==============================] - 327s 991ms/step - loss: 0.4310 - auc: 0.8790 - accuracy: 0.8305 - val_loss: 0.6276 - val_auc: 0.8611 - val_accuracy: 0.7522
Epoch 4/100
329/329 [==============================] - 326s 990ms/step - loss: 0.3972 - auc: 0.8979 - accuracy: 0.8544 - val_loss: 0.3643 - val_auc: 0.9029 - val_accuracy: 0.8658
Epoch 5/100
329/329 [==============================] - 327s 991ms/step - loss: 0.3769 - auc: 0.9083 - accuracy: 0.8606 - val_loss: 0.3312

In [ ]:
clear_train_dir()

####k1

In [ ]:
i = 1
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/100
329/329 [==============================] - 338s 969ms/step - loss: 0.5965 - auc: 0.7563 - accuracy: 0.6847 - val_loss: 0.4052 - val_auc: 0.5456 - val_accuracy: 0.8600
Epoch 2/100
329/329 [==============================] - 323s 980ms/step - loss: 0.4756 - auc: 0.8515 - accuracy: 0.7911 - val_loss: 0.3765 - val_auc: 0.6831 - val_accuracy: 0.8680
Epoch 3/100
329/329 [==============================] - 361s 1s/step - loss: 0.4317 - auc: 0.8804 - accuracy: 0.8280 - val_loss: 0.4516 - val_auc: 0.8716 - val_accuracy: 0.8243
Epoch 4/100
329/329 [==============================] - 363s 1s/step - loss: 0.4027 - auc: 0.8945 - accuracy: 0.8456 - val_loss: 0.3272 - val_auc: 0.8876 - val_accuracy: 0.8919
Epoch 5/100
329/329 [==============================] - 324s 982ms/step - lo

In [ ]:
clear_train_dir()

####k2

In [ ]:
i = 2
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 331s 985ms/step - loss: 0.6013 - auc: 0.7480 - accuracy: 0.6740 - val_loss: 0.6277 - val_auc: 0.3918 - val_accuracy: 0.6900
Epoch 2/100
329/329 [==============================] - 323s 980ms/step - loss: 0.4816 - auc: 0.8477 - accuracy: 0.7931 - val_loss: 0.9243 - val_auc: 0.6696 - val_accuracy: 0.4472
Epoch 3/100
329/329 [==============================] - 324s 982ms/step - loss: 0.4352 - auc: 0.8765 - accuracy: 0.8286 - val_loss: 0.7753 - val_auc: 0.8549 - val_accuracy: 0.6738
Epoch 4/100
329/329 [==============================] - 324s 982ms/step - loss: 0.4065 - auc: 0.8918 - accuracy: 0.8492 - val_loss: 0.3705 - val_auc: 0.9044 - val_accuracy: 0.8740
Epoch 5/100
329/329 [==============================] - 323s 979ms/step - loss: 0.3823 - auc: 0.9051 - accuracy: 0.8617 - val_loss: 0.3

In [ ]:
clear_train_dir()

####k3

In [ ]:
i = 3
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/100
329/329 [==============================] - 347s 996ms/step - loss: 0.6025 - auc: 0.7536 - accuracy: 0.6863 - val_loss: 0.4518 - val_auc: 0.6104 - val_accuracy: 0.8597
Epoch 2/100
329/329 [==============================] - 329s 998ms/step - loss: 0.4736 - auc: 0.8543 - accuracy: 0.7915 - val_loss: 0.4376 - val_auc: 0.7233 - val_accuracy: 0.8414
Epoch 3/100
329/329 [==============================] - 329s 999ms/step - loss: 0.4288 - auc: 0.8801 - accuracy: 0.8266 - val_loss: 0.6783 - val_auc: 0.8641 - val_accuracy: 0.7356
Epoch 4/100
329/329 [==============================] - 367s 1s/step - loss: 0.3951 - auc: 0.8988 - accuracy: 0.8503 - val_loss: 0.4007 - val_auc: 0.8986 - val_accuracy: 0.8571
Epoch 5/100
329/329 [==============================] - 330s 1s/step - lo

In [ ]:
clear_train_dir()

####k4

In [ ]:
i = 4
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/100
329/329 [==============================] - 348s 994ms/step - loss: 0.5957 - auc: 0.7560 - accuracy: 0.6868 - val_loss: 0.5507 - val_auc: 0.4165 - val_accuracy: 0.8146
Epoch 2/100
329/329 [==============================] - 328s 996ms/step - loss: 0.4797 - auc: 0.8499 - accuracy: 0.7878 - val_loss: 0.4367 - val_auc: 0.7314 - val_accuracy: 0.8574
Epoch 3/100
329/329 [==============================] - 329s 997ms/step - loss: 0.4391 - auc: 0.8742 - accuracy: 0.8240 - val_loss: 0.5694 - val_auc: 0.8677 - val_accuracy: 0.7782
Epoch 4/100
329/329 [==============================] - 329s 997ms/step - loss: 0.4031 - auc: 0.8941 - accuracy: 0.8478 - val_loss: 0.3742 - val_auc: 0.9042 - val_accuracy: 0.8787
Epoch 5/100
329/329 [==============================] - 329s 999ms/ste

In [ ]:
clear_train_dir()

####k5

In [ ]:
i = 5
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/100
329/329 [==============================] - 333s 953ms/step - loss: 0.5848 - auc: 0.7653 - accuracy: 0.6956 - val_loss: 0.4149 - val_auc: 0.4654 - val_accuracy: 0.8600
Epoch 2/100
329/329 [==============================] - 314s 953ms/step - loss: 0.4612 - auc: 0.8621 - accuracy: 0.8080 - val_loss: 0.3547 - val_auc: 0.7360 - val_accuracy: 0.8663
Epoch 3/100
329/329 [==============================] - 315s 956ms/step - loss: 0.4264 - auc: 0.8823 - accuracy: 0.8322 - val_loss: 0.4725 - val_auc: 0.8632 - val_accuracy: 0.8058
Epoch 4/100
329/329 [==============================] - 315s 957ms/step - loss: 0.3903 - auc: 0.9012 - accuracy: 0.8521 - val_loss: 0.3222 - val_auc: 0.8978 - val_accuracy: 0.8864
Epoch 5/100
329/329 [==============================] - 314s 952ms/ste

In [ ]:
clear_train_dir()

####k6

In [ ]:
i = 6
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/100
329/329 [==============================] - 336s 965ms/step - loss: 0.5999 - auc: 0.7522 - accuracy: 0.6812 - val_loss: 0.5080 - val_auc: 0.5207 - val_accuracy: 0.7153
Epoch 2/100
329/329 [==============================] - 318s 966ms/step - loss: 0.4771 - auc: 0.8498 - accuracy: 0.7987 - val_loss: 0.4353 - val_auc: 0.6357 - val_accuracy: 0.8608
Epoch 3/100
329/329 [==============================] - 318s 966ms/step - loss: 0.4334 - auc: 0.8774 - accuracy: 0.8321 - val_loss: 0.6357 - val_auc: 0.8455 - val_accuracy: 0.7489
Epoch 4/100
329/329 [==============================] - 319s 967ms/step - loss: 0.3988 - auc: 0.8961 - accuracy: 0.8512 - val_loss: 0.4011 - val_auc: 0.8900 - val_accuracy: 0.8593
Epoch 5/100
329/329 [==============================] - 318s 964ms/ste

In [ ]:
clear_train_dir()

####k7

In [ ]:
i = 7
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 326s 968ms/step - loss: 0.5844 - auc: 0.7677 - accuracy: 0.6956 - val_loss: 0.4470 - val_auc: 0.4491 - val_accuracy: 0.8596
Epoch 2/100
329/329 [==============================] - 318s 966ms/step - loss: 0.4665 - auc: 0.8577 - accuracy: 0.8059 - val_loss: 0.4395 - val_auc: 0.7007 - val_accuracy: 0.8423
Epoch 3/100
329/329 [==============================] - 318s 964ms/step - loss: 0.4211 - auc: 0.8837 - accuracy: 0.8372 - val_loss: 0.5903 - val_auc: 0.8490 - val_accuracy: 0.7536
Epoch 4/100
329/329 [==============================] - 318s 964ms/step - loss: 0.3948 - auc: 0.8991 - accuracy: 0.8535 - val_loss: 0.4346 - val_auc: 0.8961 - val_accuracy: 0.8436
Epoch 5/100
329/329 [==============================] - 318s 964ms/step - loss: 0.3705 - auc: 0.9111 - accuracy: 0.8638 - val_loss: 0.3

In [ ]:
clear_train_dir()

####k8

In [ ]:
i = 8
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/100
329/329 [==============================] - 334s 956ms/step - loss: 0.5767 - auc: 0.7724 - accuracy: 0.6985 - val_loss: 0.4451 - val_auc: 0.4385 - val_accuracy: 0.8600
Epoch 2/100
329/329 [==============================] - 320s 971ms/step - loss: 0.4593 - auc: 0.8632 - accuracy: 0.8117 - val_loss: 0.4165 - val_auc: 0.6126 - val_accuracy: 0.8562
Epoch 3/100
329/329 [==============================] - 321s 974ms/step - loss: 0.4205 - auc: 0.8848 - accuracy: 0.8374 - val_loss: 0.7248 - val_auc: 0.8538 - val_accuracy: 0.7141
Epoch 4/100
329/329 [==============================] - 361s 1s/step - loss: 0.3915 - auc: 0.9011 - accuracy: 0.8536 - val_loss: 0.3379 - val_auc: 0.8964 - val_accuracy: 0.8785
Epoch 5/100
329/329 [==============================] - 319s 966ms/step -

In [ ]:
clear_train_dir()

####k9

In [ ]:
i = 9
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31500 files belonging to 2 classes.
Using 21000 files for training.
Found 31500 files belonging to 2 classes.
Using 10500 files for validation.
Epoch 1/100
329/329 [==============================] - 330s 981ms/step - loss: 0.5867 - auc: 0.7630 - accuracy: 0.6955 - val_loss: 0.4120 - val_auc: 0.5452 - val_accuracy: 0.8588
Epoch 2/100
329/329 [==============================] - 321s 975ms/step - loss: 0.4713 - auc: 0.8544 - accuracy: 0.7965 - val_loss: 0.3956 - val_auc: 0.6595 - val_accuracy: 0.8593
Epoch 3/100
329/329 [==============================] - 322s 976ms/step - loss: 0.4263 - auc: 0.8802 - accuracy: 0.8253 - val_loss: 0.4989 - val_auc: 0.8595 - val_accuracy: 0.8045
Epoch 4/100
329/329 [==============================] - 362s 1s/step - loss: 0.3943 - auc: 0.8994 - accuracy: 0.8524 - val_loss: 0.3457 - val_auc: 0.8918 - val_accuracy: 0.8827
Epoch 5/100
329/329 [==============================] - 362s 1s/step - loss: 0.3756 - auc: 0.9085 - accuracy: 0.8623 - val_loss: 0.3464 - 

In [ ]:
clear_train_dir()

### Baseline: WO/ Tranfered Learning & W/ Data Aug

In [13]:
from keras.layers import RandomFlip, RandomRotation, RandomCrop

#### Declarações

In [14]:
def clear_train_dir():
  for file in os.listdir("/content/train/yes/"):
    os.remove("/content/train/yes/" + file)

  for file in os.listdir("/content/train/no/"):
    os.remove("/content/train/no/" + file)

In [15]:
def load_train_dir(ki):
  k = [0,1,2,3,4,5,6,7,8,9]
  for i in range(10):
    if i!= k[ki]:
      yesdir = "/content/jpg_k"+str(i)+"/yes/"
      nodir = "/content/jpg_k"+str(i)+"/no/"
      for file in os.listdir(yesdir):
        shutil.copy(yesdir + file, "/content/train/yes")
      for file in os.listdir(nodir):
        shutil.copy(nodir + file, "/content/train/no")

In [16]:
def train_model(w0, w1):
    train_data = image_dataset_from_directory(
      ("/content/train"),
      labels='inferred',
      validation_split=1/3,
      subset="training",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )
  
    val_data = image_dataset_from_directory(
      "/content/train",
      labels='inferred',
      validation_split=1/3,
      subset="validation",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )

    class_names = train_data.class_names

    normalization_layer = Rescaling(1./255)

    normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next(iter(normalized_data))

    normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
    val_image_batch, val_labels_batch = next(iter(normalized_val_data))

    keras.backend.clear_session()

    resnet_model = Sequential()

    resnet_model.add(RandomFlip("horizontal"))
    resnet_model.add(RandomRotation(0.2))
    resnet_model.add(RandomCrop(256, 256, seed=1)) 

    resnet = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        input_shape=image_batch.shape[1:],
        pooling='avg',
        weights=None,
        classes=2,
    )

    resnet_model.add(resnet)

    resnet_model.add(Flatten())

    resnet_model.add(Dense(512, activation="relu"))

    resnet_model.add(Dropout(0.5))

    resnet_model.add(Dense(1, activation="sigmoid"))

    resnet_model.compile(loss='binary_crossentropy', 
                         optimizer=optimizers.Adam(learning_rate=0.00001),
                         metrics=[keras.metrics.AUC(), 'accuracy'])
  
    run_hist = resnet_model.fit(
      normalized_data, 
      validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1}, 
      callbacks=[keras.callbacks.EarlyStopping(
                      monitor="val_auc",
                      min_delta=0,
                      patience=5,
                      verbose=1,
                      mode="auto",
                      baseline=None,
                      restore_best_weights=True,
                  )]
    )

    return resnet_model

In [17]:
def test_model(resnet_model, test_path):
    test_data = image_dataset_from_directory(
      test_path,
      labels='inferred',
      label_mode="int",
      batch_size = 64,
      color_mode = "rgb"
    )

    normalization_layer = Rescaling(1./255)

    normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
    test_image_batch, test_labels_batch = next(iter(normalized_test_data))

    results = resnet_model.evaluate(normalized_test_data)
    return results

In [18]:
neg = 27062
pos = 4437
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [19]:
print('Numero de imagens: NO')
!ls /content/train/no | wc -l
print('Numero de imagens: YES')
!ls /content/train/yes | wc -l

Numero de imagens: NO
0
Numero de imagens: YES
0


#### k0

In [ ]:
i = 0
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 338s 970ms/step - loss: 0.7416 - auc: 0.6241 - accuracy: 0.5732 - val_loss: 1.0299 - val_auc: 0.5323 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 320s 971ms/step - loss: 0.6526 - auc: 0.6887 - accuracy: 0.6120 - val_loss: 0.9591 - val_auc: 0.7250 - val_accuracy: 0.2462
Epoch 3/100
329/329 [==============================] - 320s 972ms/step - loss: 0.6200 - auc: 0.7206 - accuracy: 0.6353 - val_loss: 0.5859 - val_auc: 0.7441 - val_accuracy: 0.6596
Epoch 4/100
329/329 [==============================] - 320s 971ms/step - loss: 0.6061 - auc: 0.7332 - accuracy: 0.6454 - val_loss: 0.6021 - val_auc: 0.7517 - val_accuracy: 0.6583
Epoch 5/100
329/329 [==============================] - 361s 1s/step - loss: 0.5917 - auc: 0.7482 - accuracy: 0.6613 - val_loss: 0.5783

In [ ]:
clear_train_dir()

####k1

In [ ]:
i = 1
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 334s 976ms/step - loss: 0.7517 - auc: 0.6191 - accuracy: 0.5720 - val_loss: 0.7566 - val_auc: 0.5998 - val_accuracy: 0.3120
Epoch 2/100
329/329 [==============================] - 321s 973ms/step - loss: 0.6495 - auc: 0.6935 - accuracy: 0.6167 - val_loss: 0.7463 - val_auc: 0.7229 - val_accuracy: 0.5230
Epoch 3/100
329/329 [==============================] - 320s 972ms/step - loss: 0.6118 - auc: 0.7323 - accuracy: 0.6437 - val_loss: 0.6964 - val_auc: 0.7466 - val_accuracy: 0.5908
Epoch 4/100
329/329 [==============================] - 320s 972ms/step - loss: 0.6004 - auc: 0.7420 - accuracy: 0.6544 - val_loss: 0.6815 - val_auc: 0.7553 - val_accuracy: 0.6122
Epoch 5/100
329/329 [==============================] - 320s 972ms/step - loss: 0.5783 - auc: 0.7653 - accuracy: 0.6835 - val_loss: 0.8

In [ ]:
clear_train_dir()

####k2

In [ ]:
i = 2
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 328s 974ms/step - loss: 0.7429 - auc: 0.6183 - accuracy: 0.5706 - val_loss: 1.1043 - val_auc: 0.4258 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 320s 972ms/step - loss: 0.6490 - auc: 0.6890 - accuracy: 0.6070 - val_loss: 0.7517 - val_auc: 0.6486 - val_accuracy: 0.4780
Epoch 3/100
329/329 [==============================] - 320s 970ms/step - loss: 0.6182 - auc: 0.7180 - accuracy: 0.6231 - val_loss: 0.6309 - val_auc: 0.7502 - val_accuracy: 0.6222
Epoch 4/100
329/329 [==============================] - 360s 1s/step - loss: 0.6045 - auc: 0.7380 - accuracy: 0.6494 - val_loss: 0.5468 - val_auc: 0.7582 - val_accuracy: 0.7029
Epoch 5/100
329/329 [==============================] - 319s 968ms/step - loss: 0.5882 - auc: 0.7539 - accuracy: 0.6680 - val_loss: 0.7599

In [ ]:
clear_train_dir()

####k3

In [ ]:
i = 3
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 328s 975ms/step - loss: 0.7362 - auc: 0.6338 - accuracy: 0.5832 - val_loss: 0.8404 - val_auc: 0.6638 - val_accuracy: 0.2089
Epoch 2/100
329/329 [==============================] - 320s 972ms/step - loss: 0.6412 - auc: 0.6981 - accuracy: 0.6094 - val_loss: 0.8177 - val_auc: 0.7367 - val_accuracy: 0.4552
Epoch 3/100
329/329 [==============================] - 319s 969ms/step - loss: 0.6156 - auc: 0.7232 - accuracy: 0.6210 - val_loss: 0.6508 - val_auc: 0.7482 - val_accuracy: 0.6044
Epoch 4/100
329/329 [==============================] - 320s 969ms/step - loss: 0.6011 - auc: 0.7369 - accuracy: 0.6387 - val_loss: 0.6432 - val_auc: 0.7295 - val_accuracy: 0.6276
Epoch 5/100
329/329 [==============================] - 320s 970ms/step - loss: 0.5898 - auc: 0.7512 - accuracy: 0.6574 - val_loss: 0.5

In [ ]:
clear_train_dir()

####k4

In [ ]:
i = 4
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 329s 978ms/step - loss: 0.7125 - auc: 0.6326 - accuracy: 0.5794 - val_loss: 0.9410 - val_auc: 0.6437 - val_accuracy: 0.3139
Epoch 2/100
329/329 [==============================] - 321s 974ms/step - loss: 0.6327 - auc: 0.7072 - accuracy: 0.6200 - val_loss: 0.6867 - val_auc: 0.7438 - val_accuracy: 0.6042
Epoch 3/100
329/329 [==============================] - 321s 974ms/step - loss: 0.6124 - auc: 0.7269 - accuracy: 0.6373 - val_loss: 0.4943 - val_auc: 0.7333 - val_accuracy: 0.7577
Epoch 4/100
329/329 [==============================] - 321s 973ms/step - loss: 0.5987 - auc: 0.7407 - accuracy: 0.6519 - val_loss: 0.5938 - val_auc: 0.7550 - val_accuracy: 0.6604
Epoch 5/100
329/329 [==============================] - 361s 1s/step - loss: 0.5903 - auc: 0.7484 - accuracy: 0.6508 - val_loss: 0.6511

In [ ]:
clear_train_dir()

####k5

In [ ]:
i = 5
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 368s 1s/step - loss: 0.7442 - auc: 0.6266 - accuracy: 0.5771 - val_loss: 1.0545 - val_auc: 0.5291 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 320s 970ms/step - loss: 0.6530 - auc: 0.6838 - accuracy: 0.6041 - val_loss: 0.6944 - val_auc: 0.6685 - val_accuracy: 0.4961
Epoch 3/100
329/329 [==============================] - 321s 970ms/step - loss: 0.6213 - auc: 0.7149 - accuracy: 0.6181 - val_loss: 0.7277 - val_auc: 0.7499 - val_accuracy: 0.5572
Epoch 4/100
329/329 [==============================] - 320s 970ms/step - loss: 0.6021 - auc: 0.7373 - accuracy: 0.6470 - val_loss: 0.5287 - val_auc: 0.7647 - val_accuracy: 0.7115
Epoch 5/100
329/329 [==============================] - 320s 971ms/step - loss: 0.5919 - auc: 0.7496 - accuracy: 0.6584 - val_loss: 0.5459

In [ ]:
clear_train_dir()

####k6

In [ ]:
i = 6
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 335s 960ms/step - loss: 0.7506 - auc: 0.6095 - accuracy: 0.5660 - val_loss: 0.7938 - val_auc: 0.5246 - val_accuracy: 0.2846
Epoch 2/100
329/329 [==============================] - 316s 957ms/step - loss: 0.6595 - auc: 0.6783 - accuracy: 0.5983 - val_loss: 0.6831 - val_auc: 0.7369 - val_accuracy: 0.5852
Epoch 3/100
329/329 [==============================] - 316s 959ms/step - loss: 0.6230 - auc: 0.7137 - accuracy: 0.6213 - val_loss: 0.5448 - val_auc: 0.7489 - val_accuracy: 0.6854
Epoch 4/100
329/329 [==============================] - 315s 956ms/step - loss: 0.6084 - auc: 0.7306 - accuracy: 0.6441 - val_loss: 0.5817 - val_auc: 0.7647 - val_accuracy: 0.6651
Epoch 5/100
329/329 [==============================] - 315s 956ms/step - loss: 0.5886 - auc: 0.7523 - accuracy: 0.6664 - val_loss: 0.5

In [ ]:
clear_train_dir()

####k7

In [ ]:
i = 7
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 323s 961ms/step - loss: 0.7346 - auc: 0.6264 - accuracy: 0.5759 - val_loss: 0.6998 - val_auc: 0.4703 - val_accuracy: 0.6210
Epoch 2/100
329/329 [==============================] - 315s 956ms/step - loss: 0.6473 - auc: 0.6862 - accuracy: 0.6006 - val_loss: 0.7700 - val_auc: 0.7269 - val_accuracy: 0.4751
Epoch 3/100
329/329 [==============================] - 315s 956ms/step - loss: 0.6181 - auc: 0.7170 - accuracy: 0.6130 - val_loss: 0.8036 - val_auc: 0.7463 - val_accuracy: 0.5435
Epoch 4/100
329/329 [==============================] - 316s 958ms/step - loss: 0.6033 - auc: 0.7371 - accuracy: 0.6394 - val_loss: 0.7389 - val_auc: 0.7617 - val_accuracy: 0.5892
Epoch 5/100
329/329 [==============================] - 315s 957ms/step - loss: 0.5890 - auc: 0.7526 - accuracy: 0.6531 - val_loss: 0.6

In [ ]:
clear_train_dir()

####k8

In [ ]:
i = 8
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 324s 962ms/step - loss: 0.7414 - auc: 0.6216 - accuracy: 0.5712 - val_loss: 1.0316 - val_auc: 0.5914 - val_accuracy: 0.1649
Epoch 2/100
329/329 [==============================] - 315s 957ms/step - loss: 0.6413 - auc: 0.6991 - accuracy: 0.6170 - val_loss: 0.7407 - val_auc: 0.7267 - val_accuracy: 0.5898
Epoch 3/100
329/329 [==============================] - 315s 957ms/step - loss: 0.6096 - auc: 0.7310 - accuracy: 0.6378 - val_loss: 0.6252 - val_auc: 0.7590 - val_accuracy: 0.6346
Epoch 4/100
329/329 [==============================] - 316s 957ms/step - loss: 0.5925 - auc: 0.7488 - accuracy: 0.6531 - val_loss: 0.6027 - val_auc: 0.7635 - val_accuracy: 0.6521
Epoch 5/100
329/329 [==============================] - 316s 958ms/step - loss: 0.5762 - auc: 0.7666 - accuracy: 0.6840 - val_loss: 0.4

In [ ]:
clear_train_dir()

####k9

In [20]:
i = 9
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31500 files belonging to 2 classes.
Using 21000 files for training.
Found 31500 files belonging to 2 classes.
Using 10500 files for validation.
Epoch 1/100
329/329 [==============================] - 343s 981ms/step - loss: 0.7428 - auc: 0.6190 - accuracy: 0.5768 - val_loss: 0.7332 - val_auc: 0.6054 - val_accuracy: 0.3316
Epoch 2/100
329/329 [==============================] - 322s 978ms/step - loss: 0.6503 - auc: 0.6862 - accuracy: 0.6071 - val_loss: 0.7786 - val_auc: 0.7290 - val_accuracy: 0.5067
Epoch 3/100
329/329 [==============================] - 322s 978ms/step - loss: 0.6183 - auc: 0.7169 - accuracy: 0.6297 - val_loss: 0.7336 - val_auc: 0.7432 - val_accuracy: 0.5782
Epoch 4/100
329/329 [==============================] - 362s 1s/step - loss: 0.5973 - auc: 0.7432 - accuracy: 0.6514 - val_loss: 0.7063 - val_auc: 0.7585 - val_accuracy: 0.6112
Epoch 5/100
329/329 [==============================] - 362s 1s/step - loss: 0.5849 - auc: 0.7579 - accuracy: 0.6723 - val_loss: 0.7677 - 

In [21]:
clear_train_dir()